## 기본설정 및 함수정의

In [ ]:
# import pandas as pd
# import numpy as np
# import plotly.express as px
# import plotly.graph_objects as go
# import plotly.graph_objs as go
# import plotly.offline as offline
# from folium.plugins import HeatMapWithTime
# from plotly.subplots import make_subplots
# import folium
# from folium import plugins
# from folium.plugins import HeatMap
# from folium import FeatureGroup
# import json
# import math
# import re
# from datetime import datetime
# import os
# import glob
# import subprocess
# from bs4 import BeautifulSoup as bs
# from shapely.geometry import Point, Polygon, LineString
# from shapely.ops import unary_union
# import geopandas as gpd
# from geopandas import GeoSeries
# import pyproj
# from tqdm import tqdm
# from keplergl import KeplerGl

# # tqdm의 pandas전용 메소드를 호출
# tqdm.pandas()
# # 모든 열이 생략되지 않도록 설정
# pd.set_option('display.max_columns', None)

# # 영등포역 위도, 경도
# sejong = [36.51430420729354, 127.29034973889941 ]

# # Point를만드는 함수
# def make_point(x):
#     try:
#         return Point(x)
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return None
# # Polygon을 만드는 함수
# def make_pol(x):
#     try:
#         return Polygon(x[0])
#     except:
#         return Polygon(x[0][0])
    
# # Linestring을 만드는 함수
# def make_lin(x):
#     try:
#         return LineString(x)
#     except:
#         return LineString(x[0])

# # 데이터프레임을 GeoPandas 데이터프레임으로 변환하는 함수 정의
# def geo_transform(DataFrame) :
#     # csv to geopandas
#     # lon, lat data를 geometry로 변경
#     DataFrame['lat'] = DataFrame['lat'].astype(float)
#     DataFrame['lon'] = DataFrame['lon'].astype(float)
#     DataFrame['geometry'] = DataFrame.progress_apply(lambda row : Point([row['lon'], row['lat']]), axis=1) # 위도 및 경도롤 GeoPandas Point 객체로 변환
#     DataFrame = gpd.GeoDataFrame(DataFrame, geometry='geometry')
#     DataFrame.crs = {'init':'epsg:4326'} # geopandas 데이터프레임의 좌표계를 EPSG 4326으로 설정
#     DataFrame = DataFrame.to_crs({'init':'epsg:4326'}) # 데이터프레임의 좌표계를 자체 좌표계에서 EPSG 4326으로 변환
#     return DataFrame

#### 세종시 행정경계(대상구역)

In [ ]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/11.세종시_대상구역도.geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
object_area_df = pd.json_normalize(geojson_data['features'])
object_area_df['geometry'] = object_area_df['geometry.coordinates'].apply(lambda x : make_pol(x))
object_area_df.drop(columns="geometry.coordinates", inplace=True)
object_area_df.rename(columns = {'properties.EMD_NM' : 'ADM_NM'}, inplace = True)
object_area_df = object_area_df[['ADM_NM', 'geometry']]

# shp to geodataframe convert
shapefile_path = "SBJ_2406_002/_census_data_2023_bnd_dong_bnd_dong_29010_2023_2023"
sejong_gdf = gpd.read_file(shapefile_path)
sejong_gdf = sejong_gdf.to_crs(epsg=4326) #EPSG4326 형식으로 변환
sejong_gdf = sejong_gdf[['ADM_NM', 'geometry']]
sejong_gdf = sejong_gdf[~sejong_gdf['ADM_NM'].str.endswith('동')]
sejong_gdf =  pd.concat([object_area_df, sejong_gdf], ignore_index=True)
# sejong_gdf 데이터프레임을 GeoDataFrame으로 변환
sejong_gdf = gpd.GeoDataFrame(sejong_gdf, geometry='geometry')
# 조건에 따라 'newtown' 열 생성 및 값 지정
sejong_gdf['newtown'] = sejong_gdf['ADM_NM'].apply(lambda x: '신도시' if x.endswith('동') else '그외지역')

#### 격자(매핑용)

In [5]:
# GeoJSON 파일 불러오기
with open('SBJ_2406_002/4.세종시_격자(매핑용).geojson', encoding="UTF8") as geojson_file:
    geojson_data = json.load(geojson_file)
grid_map_df = pd.json_normalize(geojson_data['features'])
grid_map_df['geometry'] = grid_map_df['geometry.coordinates'].apply(lambda x : make_pol(x))
grid_map_df.drop(columns="geometry.coordinates", axis=1, inplace=True)
# grid_map_df 데이터프레임을 GeoDataFrame으로 변환
grid_map_df = gpd.GeoDataFrame(grid_map_df, geometry='geometry')

##### 공실률 계산

In [48]:
import numpy as np
import pandas as pd
# 모든 열이 생략되지 않도록 설정
pd.set_option('display.max_columns', None)

data = {
    "Year": ["2018", "2019", "2020", "2021", "2022", "2023"],
    "고운동": [38.72, 33.67, 38.85, 25.90, 24.72, 30.52],
    "아름동": [38.57, 36.77, 45.45, 34.65, 37.42, 23.4],
    "종촌동": [15.30, 19.83, 17.7, 22.00, 13.49, 12.00],
    "도담동": [13.83, 12.20, 21.7, 12.75, 12.07, 15.7],
    "어진동": [15.80, 14.87, 26.00, 17.35, 24.50, 23.40],
    "다정동": [75.03, 58.87, 40.7, 44.85, 17.45, 11.9],
    "새롬동": [20.97, 11.40, 13.20, 1.35, 8.03, 4.1],
    "한솔동": [43.63, 54.53, 30.10, 59.55, 56.25, 30.10],
    "대평동": [58.83, 57.0, 49.3, 66.30, 55.42, 60.2],
    "보람동": [57.98, 52.85, 61.35, 50.80, 21.23, 24.35],
    "소담동": [54.03, 38.47, 55.1, 32.60, 44.52, 35.5],
    "나성동": [58.83, 52.96, 42.13, None, None, 36],
    "반곡동": [None, 79.6, 55.1, None, None, 32.5],
    "해밀동": [None, None, None, None, None, 32.5]
}

# Create the dataframe
empty_rate_df = pd.DataFrame(data)
empty_rate_df = empty_rate_df.iloc[2:,:]
# Filling NaN values with the mean of the respective columns
empty_rate_df = empty_rate_df.apply(lambda x: x.fillna(x.mean()), axis=0)
empty_rate_df

,Year,고운동,아름동,종촌동,도담동,어진동,다정동,새롬동,한솔동,대평동,보람동,소담동,나성동,반곡동,해밀동
2,2020,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5
3,2021,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5
4,2022,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5
5,2023,30.52,23.40,12.00,15.70,23.40,11.90,4.10,30.10,60.20,24.35,35.50,36.000,32.5,32.5


In [76]:
import plotly.graph_objects as go

# Define the years and empty rates for each area
years = empty_rate_df['Year'].tolist()
areas = empty_rate_df.columns[1:]

# Create a figure with a line plot for each area
fig = go.Figure()

# Sorting the areas by the average empty rate descending
area_means = empty_rate_df.drop(columns='Year').mean().sort_values(ascending=False)
sorted_areas = area_means.index

for area in sorted_areas:
    fig.add_trace(go.Scatter(
        x=years, 
        y=empty_rate_df[area].tolist(),
        mode='lines+markers',
        name=area
    ))
# 색상 업데이트
colors =px.colors.sequential.Plasma # 컬러맵
for i, trace in enumerate(fig.data):
    trace.line.color = colors[i % len(colors)]

# Customize the layout
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Empty Rate (%)",
    legend_title="행정동",
    hovermode="x unified"
)

# Show the plot
fig.show()

##### 공실률 - 거주인구 상관관계

In [49]:
pop_trend_df = pd.read_csv("행정동별_거주인구.csv")
pop_trend_df = pop_trend_df[pop_trend_df['ADM_NM'].isin(empty_rate_df.columns[1:])]
pop_pivot_df = pop_trend_df.pivot(index='year', columns='ADM_NM', values='인구')
pop_pivot_df = pop_pivot_df[empty_rate_df.columns[1:]]

# Now, we merge the population data with the existing vacancy rate dataframe (df)
empty_rate_df['Year'] = empty_rate_df['Year'].astype(int)  # Ensure the 'Year' column in df is of integer type for merging
merged_respop_df = empty_rate_df.merge(pop_pivot_df, left_on='Year', right_index=True, how='left')

# Rename population columns for clarity (e.g., adding "_인구" suffix)
merged_respop_df = merged_respop_df.rename(columns={f"{name}_y": f"{name}_인구" for name in pop_pivot_df.columns})

merged_respop_df

,Year,고운동_x,아름동_x,종촌동_x,도담동_x,어진동_x,다정동_x,새롬동_x,한솔동_x,대평동_x,보람동_x,소담동_x,나성동_x,반곡동_x,해밀동_x,고운동_인구,아름동_인구,종촌동_인구,도담동_인구,어진동_인구,다정동_인구,새롬동_인구,한솔동_인구,대평동_인구,보람동_인구,소담동_인구,나성동_인구,반곡동_인구,해밀동_인구
2,2020,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5,23474,17987,20567,18915,6291,19325,19813,14220,7933,15837,13759,5175,9134,2478
3,2021,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5,23384,18055,20540,18883,6539,21357,19606,14038,8263,15758,15099,5966,9052,5526
4,2022,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5,23726,18204,20464,18901,6455,21301,19565,13762,8119,15581,14858,5980,9007,5548
5,2023,30.52,23.40,12.00,15.70,23.40,11.90,4.10,30.10,60.20,24.35,35.50,36.000,32.5,32.5,23869,18581,20572,18921,6405,21556,19884,13976,8230,15798,15189,5753,9224,5580


In [50]:
# Calculating the correlation between vacancy rates and population data
# We'll focus on the columns that have both vacancy rate and population data available

# Extracting relevant columns for correlation analysis
vacancy_columns = merged_respop_df.columns[1:15].tolist()
population_columns = merged_respop_df.columns[15:].tolist()

# Create a DataFrame with only the columns needed for correlation
correlation_df = merged_respop_df[vacancy_columns + population_columns]

# Calculate correlation matrix
correlation_matrix = correlation_df.corr()

# Extracting the correlation between vacancy rates and population data
# The first half of the columns are vacancy rates, the second half are populations
correlation_vacancy_population = correlation_matrix.loc[vacancy_columns, population_columns]

correlation_vacancy_population

,고운동_인구,아름동_인구,종촌동_인구,도담동_인구,어진동_인구,다정동_인구,새롬동_인구,한솔동_인구,대평동_인구,보람동_인구,소담동_인구,나성동_인구,반곡동_인구,해밀동_인구
고운동_x,-0.175478,-0.218119,0.685059,0.640723,-0.914155,-0.874724,0.733752,0.835500,-0.761099,0.761060,-0.830923,-0.991544,0.572943,-0.915923
아름동_x,-0.664905,-0.907537,-0.197248,-0.179762,-0.399644,-0.812514,-0.296879,0.384834,-0.809736,-0.017283,-0.846829,-0.536570,-0.495248,-0.754878
종촌동_x,-0.965820,-0.773592,0.168839,-0.740400,0.331292,-0.251172,-0.375744,0.508045,0.041413,0.285409,-0.178325,-0.045819,-0.385498,-0.220050
도담동_x,-0.193036,-0.246288,0.663258,0.626819,-0.916388,-0.890679,0.710756,0.832673,-0.781055,0.744507,-0.849605,-0.995396,0.544732,-0.928963
어진동_x,0.451287,0.113751,-0.041211,0.804549,-0.856899,-0.542315,0.425974,0.036249,-0.775697,-0.018167,-0.615805,-0.611295,0.263286,-0.551430
다정동_x,-0.989922,-0.865013,0.260574,-0.550122,0.045278,-0.523648,-0.226017,0.678715,-0.241096,0.406258,-0.455356,-0.324687,-0.297411,-0.495954
새롬동_x,-0.056095,-0.388893,-0.012074,0.520828,-0.868524,-0.858687,0.242314,0.333538,-0.986400,0.098797,-0.914192,-0.790535,0.021840,-0.843737
한솔동_x,-0.351896,-0.354027,-0.726887,-0.921935,0.854763,0.490726,-0.964129,-0.553124,0.456289,-0.699838,0.452382,0.775651,-0.883713,0.564047
대평동_x,-0.086732,0.258267,0.026213,-0.613351,0.887139,0.791214,-0.296451,-0.254089,0.953985,-0.062974,0.854019,0.758561,-0.086167,0.782227
보람동_x,-0.873589,-0.772210,0.505648,-0.199404,-0.307265,-0.751470,0.118437,0.889455,-0.466838,0.650026,-0.674289,-0.644338,0.003915,-0.747567


In [51]:
import plotly.express as px

# Create a correlation heatmap using Plotly
fig = px.imshow(
    correlation_vacancy_population,
    labels=dict(x="Population Data", y="Vacancy Rates", color="Correlation"),
    x=correlation_vacancy_population.columns,
    y=correlation_vacancy_population.index,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1
)

fig.show()

##### 공실률 - 유동인구 상관관계

In [52]:
fpop_trend_df = pd.read_csv("행정동별_유동인구.csv")
fpop_trend_df = pop_trend_df[pop_trend_df['ADM_NM'].isin(empty_rate_df.columns[1:])]
fpop_trend_df.rename(columns={"인구":"유동인구"}, inplace=True)

In [53]:
# Pivot the floating population data to match the main dataframe format
fpop_pivot_df = fpop_trend_df.pivot(index='year', columns='ADM_NM', values='유동인구')

# Merge the floating population data with the main dataframe
empty_rate_df['Year'] = empty_rate_df['Year'].astype(int)
merged_fpop_df = empty_rate_df.merge(fpop_pivot_df, left_on='Year', right_index=True, how='left')

# Rename the floating population columns for clarity (e.g., adding "_유동인구" suffix)
merged_fpop_df = merged_fpop_df.rename(columns={f"{name}_y": f"{name}_유동인구" for name in fpop_pivot_df.columns})

In [54]:
# Extract the correlation between vacancy rates and floating population data
vacancy_columns = ['고운동_x', '아름동_x', '종촌동_x', '도담동_x', '어진동_x', '다정동_x', '새롬동_x', '한솔동_x', '대평동_x', '보람동_x', '소담동_x', '나성동_x', '반곡동_x', '해밀동_x']
fpop_columns = [f"{name}_유동인구" for name in fpop_pivot_df.columns]

# Calculate correlation matrix
correlation_matrix_fpop = merged_fpop_df[vacancy_columns + fpop_columns].corr()

# Extracting the correlation between vacancy rates and floating population data
correlation_vacancy_fpop = correlation_matrix_fpop.loc[vacancy_columns, fpop_columns]

correlation_vacancy_fpop

,고운동_유동인구,나성동_유동인구,다정동_유동인구,대평동_유동인구,도담동_유동인구,반곡동_유동인구,보람동_유동인구,새롬동_유동인구,소담동_유동인구,아름동_유동인구,어진동_유동인구,종촌동_유동인구,한솔동_유동인구,해밀동_유동인구
고운동_x,-0.175478,-0.991544,-0.874724,-0.761099,0.640723,0.572943,0.761060,0.733752,-0.830923,-0.218119,-0.914155,0.685059,0.835500,-0.915923
아름동_x,-0.664905,-0.536570,-0.812514,-0.809736,-0.179762,-0.495248,-0.017283,-0.296879,-0.846829,-0.907537,-0.399644,-0.197248,0.384834,-0.754878
종촌동_x,-0.965820,-0.045819,-0.251172,0.041413,-0.740400,-0.385498,0.285409,-0.375744,-0.178325,-0.773592,0.331292,0.168839,0.508045,-0.220050
도담동_x,-0.193036,-0.995396,-0.890679,-0.781055,0.626819,0.544732,0.744507,0.710756,-0.849605,-0.246288,-0.916388,0.663258,0.832673,-0.928963
어진동_x,0.451287,-0.611295,-0.542315,-0.775697,0.804549,0.263286,-0.018167,0.425974,-0.615805,0.113751,-0.856899,-0.041211,0.036249,-0.551430
다정동_x,-0.989922,-0.324687,-0.523648,-0.241096,-0.550122,-0.297411,0.406258,-0.226017,-0.455356,-0.865013,0.045278,0.260574,0.678715,-0.495954
새롬동_x,-0.056095,-0.790535,-0.858687,-0.986400,0.520828,0.021840,0.098797,0.242314,-0.914192,-0.388893,-0.868524,-0.012074,0.333538,-0.843737
한솔동_x,-0.351896,0.775651,0.490726,0.456289,-0.921935,-0.883713,-0.699838,-0.964129,0.452382,-0.354027,0.854763,-0.726887,-0.553124,0.564047
대평동_x,-0.086732,0.758561,0.791214,0.953985,-0.613351,-0.086167,-0.062974,-0.296451,0.854019,0.258267,0.887139,0.026213,-0.254089,0.782227
보람동_x,-0.873589,-0.644338,-0.751470,-0.466838,-0.199404,0.003915,0.650026,0.118437,-0.674289,-0.772210,-0.307265,0.505648,0.889455,-0.747567


In [55]:
# Importing plotly express for visualization
import plotly.express as px

# Creating a heatmap using Plotly Express to visualize the correlation between vacancy rates and floating population
fig = px.imshow(
    correlation_vacancy_fpop,
    labels=dict(x="Floating Population Data", y="Vacancy Rates", color="Correlation"),
    x=correlation_vacancy_fpop.columns,
    y=correlation_vacancy_fpop.index,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1
)

fig.show()

##### 공실률 - 상가폐업률 상관관계

In [77]:
closure_rates = pd.read_csv("행정동별_상가폐업률.csv")
col_lst = empty_rate_df.columns[1:].tolist()
col_lst = ['Year'] + col_lst
closure_rates = closure_rates[col_lst]

In [78]:
closure_rates

,Year,고운동,아름동,종촌동,도담동,어진동,다정동,새롬동,한솔동,대평동,보람동,소담동,나성동,반곡동,해밀동
0,2020,6.571429,6.519559,8.105263,9.036743,5.385330,5.619413,7.354555,7.155635,9.852217,6.342183,4.489338,5.471956,2.034429,0.549451
1,2021,7.822410,7.722772,9.907121,8.673978,4.632663,7.393484,8.559047,9.106830,9.703947,6.617647,7.500000,7.496653,3.692308,2.688172
2,2022,8.403361,7.631318,7.619048,7.568113,6.039650,11.177885,11.064278,6.486486,12.160000,7.636364,6.380638,12.457770,5.722892,5.235602
3,2023,10.594668,9.426628,12.525050,10.981535,11.120802,11.708483,9.925293,8.626761,10.876623,8.829864,9.564293,10.530859,6.427504,7.653061


In [79]:
# Aligning both DataFrames on 'Year' column for correlation analysis
merged_closure_rates_df = pd.merge(closure_rates, empty_rate_df, on='Year', how='inner')

# Rename the floating population columns for clarity (e.g., adding "_유동인구" suffix)
merged_closure_rates_df = merged_closure_rates_df.rename(columns={f"{name}_y": f"{name}_폐업률" for name in fpop_pivot_df.columns})

# Extract the correlation between vacancy rates and floating population data
vacancy_columns = ['고운동_x', '아름동_x', '종촌동_x', '도담동_x', '어진동_x', '다정동_x', '새롬동_x', '한솔동_x', '대평동_x', '보람동_x', '소담동_x', '나성동_x', '반곡동_x', '해밀동_x']
closure_rates_columns = [f"{name}_폐업률" for name in closure_rates.columns[1:]]

# Dropping the 'Year' column and calculating the correlation matrix
correlation_matrix = merged_closure_rates_df.drop(columns=['Year']).corr()

# Extracting the correlation between closure rates and vacancy rates
closure_vacancy_corr = correlation_matrix.loc[vacancy_columns, closure_rates_columns]

closure_vacancy_corr

,고운동_폐업률,아름동_폐업률,종촌동_폐업률,도담동_폐업률,어진동_폐업률,다정동_폐업률,새롬동_폐업률,한솔동_폐업률,대평동_폐업률,보람동_폐업률,소담동_폐업률,나성동_폐업률,반곡동_폐업률,해밀동_폐업률
고운동_x,-0.392463,-0.963054,-0.673466,-0.420459,-0.072013,-0.820600,-0.558848,-0.172657,0.437008,-0.801606,-0.641043,-0.976754,-0.976754,NaN
아름동_x,-0.402182,-0.994862,-0.540096,-0.432252,-0.218853,-0.709400,-0.667598,-0.139377,0.560100,-0.709690,-0.740829,-0.989308,-0.989308,NaN
종촌동_x,-0.058722,-0.895293,-0.245130,-0.092983,-0.319113,-0.363006,-0.630419,-0.366928,0.563247,-0.283924,-0.688105,-0.812542,-0.812542,NaN
도담동_x,0.374895,-0.668531,-0.328079,0.342713,0.039896,-0.313287,-0.230791,-0.737274,0.165119,-0.079851,-0.298700,-0.558283,-0.558283,NaN
어진동_x,0.046461,-0.820018,-0.774666,0.017110,0.256457,-0.798476,-0.216933,-0.586978,0.090905,-0.614377,-0.314750,-0.796194,-0.796194,NaN
다정동_x,-0.589371,-0.754710,-0.794659,-0.604318,0.097312,-0.937049,-0.385135,0.077482,0.257104,-0.990077,-0.460440,-0.843012,-0.843012,NaN
새롬동_x,-0.746460,-0.520510,-0.669304,-0.749737,0.064820,-0.823372,-0.319926,0.364666,0.216032,-0.969107,-0.369799,-0.650230,-0.650230,NaN
한솔동_x,-0.156455,-0.595268,0.402841,-0.181062,-0.781421,0.234973,-0.795198,0.028571,0.816823,0.167418,-0.786815,-0.489065,-0.489065,NaN
대평동_x,-0.505739,-0.224779,-0.764977,-0.500487,0.410790,-0.814218,0.073925,0.213848,-0.175622,-0.880342,0.022407,-0.368553,-0.368553,NaN
보람동_x,-0.306694,-0.866141,-0.836168,-0.331101,0.166574,-0.932362,-0.350012,-0.264724,0.213427,-0.866917,-0.442922,-0.899556,-0.899556,NaN


In [80]:
# Importing plotly express for visualization
import plotly.express as px

# Creating a heatmap using Plotly Express to visualize the correlation between vacancy rates and floating population
fig = px.imshow(
    closure_vacancy_corr,
    labels=dict(x="폐업률", y="Vacancy Rates", color="Correlation"),
    x=closure_vacancy_corr.columns,
    y=closure_vacancy_corr.index,
    color_continuous_scale='RdBu',
    zmin=-1, zmax=1
)

fig.show()

##### 공실률 - 매출 상관관계

In [68]:
sales_sum = pd.read_csv('행정동별_총매출.csv')
sales_mean = pd.read_csv('행정동별_평균매출.csv')

In [87]:
# Flatten the empty_rate_df to have each row correspond to a single year's data per area
flattened_empty_rate_df = empty_rate_df.melt(id_vars=["Year"], var_name="ADM_NM", value_name="Empty_Rate")

# Merge with sales_sum to align sales data with each row of empty rate data
merged_data = pd.merge(flattened_empty_rate_df, sales_sum, on="ADM_NM")

# Calculating the correlation for each year
correlation_per_year = merged_data.groupby('Year').apply(lambda x: x['Empty_Rate'].corr(x['EST_SALES']))

correlation_per_year

Year
2020   -0.121273
2021   -0.289819
2022   -0.225108
2023   -0.196593
dtype: float64

In [89]:
# Flatten the empty_rate_df to have each row correspond to a single year's data per area
flattened_empty_rate_df = empty_rate_df.melt(id_vars=["Year"], var_name="ADM_NM", value_name="Empty_Rate")

# Merge with sales_sum to align sales data with each row of empty rate data
merged_data = pd.merge(flattened_empty_rate_df, sales_mean, on="ADM_NM")

# Calculating the correlation for each year
correlation_per_year = merged_data.groupby('Year').apply(lambda x: x['Empty_Rate'].corr(x['EST_SALES']))

correlation_per_year

Year
2020   -0.416689
2021   -0.287671
2022    0.021375
2023   -0.154180
dtype: float64

In [172]:
# Flatten the empty_rate_df to merge with sales_mean
flattened_empty_rate_df = empty_rate_df.melt(id_vars=['Year'], var_name='ADM_NM', value_name='공실률')

# Merge with sales_mean
merged_sales_mean_df = pd.merge(flattened_empty_rate_df, sales_mean, on='ADM_NM', how='left')

# Reshape data back to wide format
merged_sales_mean_df = merged_sales_mean_df.pivot_table(index='Year', columns='ADM_NM', values='공실률').reset_index()

# Adding the sales mean data as additional columns
for index, row in sales_mean.iterrows():
    merged_sales_mean_df[row['ADM_NM'] + '_매출평균'] = row['EST_SALES']

merged_sales_mean_df.fillna(0, inplace=True)  # Fill NaN if any
merged_sales_mean_df = pd.DataFrame(merged_sales_mean_df.to_dict())
merged_sales_mean_df

,Year,고운동,나성동,다정동,대평동,도담동,반곡동,보람동,새롬동,소담동,아름동,어진동,종촌동,한솔동,해밀동,가람동_매출평균,고운동_매출평균,나성동_매출평균,누리동_매출평균,다솜동_매출평균,다정동_매출평균,대평동_매출평균,도담동_매출평균,반곡동_매출평균,보람동_매출평균,산울동_매출평균,새롬동_매출평균,세종동_매출평균,소담동_매출평균,아름동_매출평균,어진동_매출평균,종촌동_매출평균,집현동_매출평균,한별동_매출평균,한솔동_매출평균,합강동_매출평균,해밀동_매출평균
0,2020,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
1,2021,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
2,2022,24.72,39.065,17.45,55.42,12.07,43.8,21.23,8.03,44.52,37.42,24.50,13.49,56.25,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
3,2023,30.52,36.000,11.90,60.20,15.70,32.5,24.35,4.10,35.50,23.40,23.40,12.00,30.10,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514


In [171]:
# Flatten the empty_rate_df to merge with sales_sum
flattened_empty_rate_df = empty_rate_df.melt(id_vars=['Year'], var_name='ADM_NM', value_name='공실률')

# Merge with sales_sum
merged_sales_mean_df = pd.merge(flattened_empty_rate_df, sales_sum, on='ADM_NM', how='left')

# Reshape data back to wide format
merged_sales_sum_df = merged_sales_mean_df.pivot_table(index='Year', columns='ADM_NM', values='공실률').reset_index()

# Adding the sales sum data as additional columns
for index, row in sales_sum.iterrows():
    merged_sales_sum_df[row['ADM_NM'] + '_총매출'] = row['EST_SALES']

merged_sales_sum_df.fillna(0, inplace=True)  # Fill NaN if any
merged_sales_sum_df = pd.DataFrame(merged_sales_sum_df.to_dict())
merged_sales_sum_df

,Year,고운동,나성동,다정동,대평동,도담동,반곡동,보람동,새롬동,소담동,아름동,어진동,종촌동,한솔동,해밀동,가람동_총매출,고운동_총매출,나성동_총매출,누리동_총매출,다솜동_총매출,다정동_총매출,대평동_총매출,도담동_총매출,반곡동_총매출,보람동_총매출,산울동_총매출,새롬동_총매출,세종동_총매출,소담동_총매출,아름동_총매출,어진동_총매출,종촌동_총매출,집현동_총매출,한별동_총매출,한솔동_총매출,합강동_총매출,해밀동_총매출
0,2020,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
1,2021,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
2,2022,24.72,39.065,17.45,55.42,12.07,43.8,21.23,8.03,44.52,37.42,24.50,13.49,56.25,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
3,2023,30.52,36.000,11.90,60.20,15.70,32.5,24.35,4.10,35.50,23.40,23.40,12.00,30.10,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0


In [193]:
merged_fpop_df

,Year,고운동_x,아름동_x,종촌동_x,도담동_x,어진동_x,다정동_x,새롬동_x,한솔동_x,대평동_x,보람동_x,소담동_x,나성동_x,반곡동_x,해밀동_x,고운동_유동인구,나성동_유동인구,다정동_유동인구,대평동_유동인구,도담동_유동인구,반곡동_유동인구,보람동_유동인구,새롬동_유동인구,소담동_유동인구,아름동_유동인구,어진동_유동인구,종촌동_유동인구,한솔동_유동인구,해밀동_유동인구
2,2020,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5,23474,5175,19325,7933,18915,9134,15837,19813,13759,17987,6291,20567,14220,2478
3,2021,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5,23384,5966,21357,8263,18883,9052,15758,19606,15099,18055,6539,20540,14038,5526
4,2022,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5,23726,5980,21301,8119,18901,9007,15581,19565,14858,18204,6455,20464,13762,5548
5,2023,30.52,23.40,12.00,15.70,23.40,11.90,4.10,30.10,60.20,24.35,35.50,36.000,32.5,32.5,23869,5753,21556,8230,18921,9224,15798,19884,15189,18581,6405,20572,13976,5580


In [194]:
merged_closure_rates_df

,Year,고운동_x,아름동_x,종촌동_x,도담동_x,어진동_x,다정동_x,새롬동_x,한솔동_x,대평동_x,보람동_x,소담동_x,나성동_x,반곡동_x,해밀동_x,고운동_폐업률,아름동_폐업률,종촌동_폐업률,도담동_폐업률,어진동_폐업률,다정동_폐업률,새롬동_폐업률,한솔동_폐업률,대평동_폐업률,보람동_폐업률,소담동_폐업률,나성동_폐업률,반곡동_폐업률,해밀동_폐업률
0,2020,6.571429,6.519559,8.105263,9.036743,5.385330,5.619413,7.354555,7.155635,9.852217,6.342183,4.489338,5.471956,2.034429,0.549451,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5
1,2021,7.822410,7.722772,9.907121,8.673978,4.632663,7.393484,8.559047,9.106830,9.703947,6.617647,7.500000,7.496653,3.692308,2.688172,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5
2,2022,8.403361,7.631318,7.619048,7.568113,6.039650,11.177885,11.064278,6.486486,12.160000,7.636364,6.380638,12.457770,5.722892,5.235602,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5
3,2023,10.594668,9.426628,12.525050,10.981535,11.120802,11.708483,9.925293,8.626761,10.876623,8.829864,9.564293,10.530859,6.427504,7.653061,30.52,23.40,12.00,15.70,23.40,11.90,4.10,30.10,60.20,24.35,35.50,36.000,32.5,32.5


In [196]:
merged_sales_sum_df

,Year,고운동,나성동,다정동,대평동,도담동,반곡동,보람동,새롬동,소담동,아름동,어진동,종촌동,한솔동,해밀동,가람동_총매출,고운동_총매출,나성동_총매출,누리동_총매출,다솜동_총매출,다정동_총매출,대평동_총매출,도담동_총매출,반곡동_총매출,보람동_총매출,산울동_총매출,새롬동_총매출,세종동_총매출,소담동_총매출,아름동_총매출,어진동_총매출,종촌동_총매출,집현동_총매출,한별동_총매출,한솔동_총매출,합강동_총매출,해밀동_총매출
0,2020,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
1,2021,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
2,2022,24.72,39.065,17.45,55.42,12.07,43.8,21.23,8.03,44.52,37.42,24.50,13.49,56.25,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0
3,2023,30.52,36.000,11.90,60.20,15.70,32.5,24.35,4.10,35.50,23.40,23.40,12.00,30.10,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0


In [195]:
merged_sales_mean_df

,Year,고운동,나성동,다정동,대평동,도담동,반곡동,보람동,새롬동,소담동,아름동,어진동,종촌동,한솔동,해밀동,가람동_매출평균,고운동_매출평균,나성동_매출평균,누리동_매출평균,다솜동_매출평균,다정동_매출평균,대평동_매출평균,도담동_매출평균,반곡동_매출평균,보람동_매출평균,산울동_매출평균,새롬동_매출평균,세종동_매출평균,소담동_매출평균,아름동_매출평균,어진동_매출평균,종촌동_매출평균,집현동_매출평균,한별동_매출평균,한솔동_매출평균,합강동_매출평균,해밀동_매출평균
0,2020,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
1,2021,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
2,2022,24.72,39.065,17.45,55.42,12.07,43.8,21.23,8.03,44.52,37.42,24.50,13.49,56.25,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
3,2023,30.52,36.000,11.90,60.20,15.70,32.5,24.35,4.10,35.50,23.40,23.40,12.00,30.10,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514


In [187]:
merged_df = pd.merge(merged_fpop_df, merged_respop_df, on='Year')
merged_df = pd.merge(merged_df, merged_closure_rates_df, on='Year')
merged_df = pd.merge(merged_df, merged_sales_sum_df, on='Year', how='left')
merged_df = pd.merge(merged_df, merged_sales_mean_df, on='Year', suffixes=('_sum', '_mean'))

In [188]:
merged_df

,Year,고운동_x_x,아름동_x_x,종촌동_x_x,도담동_x_x,어진동_x_x,다정동_x_x,새롬동_x_x,한솔동_x_x,대평동_x_x,보람동_x_x,소담동_x_x,나성동_x_x,반곡동_x_x,해밀동_x_x,고운동_유동인구,나성동_유동인구,다정동_유동인구,대평동_유동인구,도담동_유동인구,반곡동_유동인구,보람동_유동인구,새롬동_유동인구,소담동_유동인구,아름동_유동인구,어진동_유동인구,종촌동_유동인구,한솔동_유동인구,해밀동_유동인구,고운동_x_y,아름동_x_y,종촌동_x_y,도담동_x_y,어진동_x_y,다정동_x_y,새롬동_x_y,한솔동_x_y,대평동_x_y,보람동_x_y,소담동_x_y,나성동_x_y,반곡동_x_y,해밀동_x_y,고운동_인구,아름동_인구,종촌동_인구,도담동_인구,어진동_인구,다정동_인구,새롬동_인구,한솔동_인구,대평동_인구,보람동_인구,소담동_인구,나성동_인구,반곡동_인구,해밀동_인구,고운동_x,아름동_x,종촌동_x,도담동_x,어진동_x,다정동_x,새롬동_x,한솔동_x,대평동_x,보람동_x,소담동_x,나성동_x,반곡동_x,해밀동_x,고운동_폐업률,아름동_폐업률,종촌동_폐업률,도담동_폐업률,어진동_폐업률,다정동_폐업률,새롬동_폐업률,한솔동_폐업률,대평동_폐업률,보람동_폐업률,소담동_폐업률,나성동_폐업률,반곡동_폐업률,해밀동_폐업률,고운동_sum,나성동_sum,다정동_sum,대평동_sum,도담동_sum,반곡동_sum,보람동_sum,새롬동_sum,소담동_sum,아름동_sum,어진동_sum,종촌동_sum,한솔동_sum,해밀동_sum,가람동_총매출,고운동_총매출,나성동_총매출,누리동_총매출,다솜동_총매출,다정동_총매출,대평동_총매출,도담동_총매출,반곡동_총매출,보람동_총매출,산울동_총매출,새롬동_총매출,세종동_총매출,소담동_총매출,아름동_총매출,어진동_총매출,종촌동_총매출,집현동_총매출,한별동_총매출,한솔동_총매출,합강동_총매출,해밀동_총매출,고운동_mean,나성동_mean,다정동_mean,대평동_mean,도담동_mean,반곡동_mean,보람동_mean,새롬동_mean,소담동_mean,아름동_mean,어진동_mean,종촌동_mean,한솔동_mean,해밀동_mean,가람동_매출평균,고운동_매출평균,나성동_매출평균,누리동_매출평균,다솜동_매출평균,다정동_매출평균,대평동_매출평균,도담동_매출평균,반곡동_매출평균,보람동_매출평균,산울동_매출평균,새롬동_매출평균,세종동_매출평균,소담동_매출평균,아름동_매출평균,어진동_매출평균,종촌동_매출평균,집현동_매출평균,한별동_매출평균,한솔동_매출평균,합강동_매출평균,해밀동_매출평균
0,2020,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5,23474,5175,19325,7933,18915,9134,15837,19813,13759,17987,6291,20567,14220,2478,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5,23474,17987,20567,18915,6291,19325,19813,14220,7933,15837,13759,5175,9134,2478,6.571429,6.519559,8.105263,9.036743,5.385330,5.619413,7.354555,7.155635,9.852217,6.342183,4.489338,5.471956,2.034429,0.549451,38.85,45.45,17.70,21.70,26.00,40.70,13.20,30.10,49.30,61.35,55.10,42.130,55.1,32.5,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0,38.85,42.130,40.70,49.30,21.70,55.1,61.35,13.20,55.10,45.45,26.00,17.70,30.10,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
1,2021,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5,23384,5966,21357,8263,18883,9052,15758,19606,15099,18055,6539,20540,14038,5526,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5,23384,18055,20540,18883,6539,21357,19606,14038,8263,15758,15099,5966,9052,5526,7.822410,7.722772,9.907121,8.673978,4.632663,7.393484,8.559047,9.106830,9.703947,6.617647,7.500000,7.496653,3.692308,2.688172,25.90,34.65,22.00,12.75,17.35,44.85,1.35,59.55,66.30,50.80,32.60,39.065,43.8,32.5,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,7951753.0,82493380.0,184444748.0,13167.0,0.0,53051458.0,41695736.0,83618675.0,37869789.0,81744701.0,10780173.0,118263554.0,19036516.0,58703995.0,86855772.0,66472173.0,61676660.0,6979024.0,133915.0,52452261.0,174982.0,11743023.0,25.90,39.065,44.85,66.30,12.75,43.8,50.80,1.35,32.60,34.65,17.35,22.00,59.55,32.5,76459.163462,14306.864377,19036.510269,2194.5,0.0,15080.005117,16351.26902,16470.095529,17379.435062,13878.557046,49908.208333,19766.597693,13646.248029,14276.263375,19005.639387,16684.78238,16295.022457,11095.427663,11159.583333,16725.848533,29163.666667,17345.676514
2,2022,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5,23726,5980,21301,8119,18901,9007,15581,19565,14858,18204,6455,20464,13762,5548,24.72,37.42,13.49,12.07,24.50,17.45,8.03,56.25,55.42,21.23,44.52,39.065,43.8,32.5,23726,18204,20464,18901,6455,21301,19565,13762,8119,15581,14858,5980,9007,5548,8.403361

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 데이터 전처리
merged_df = merged_df.fillna(0)  # 결측치를 0으로 채움
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_df.drop(columns=['Year']))

# 입력 데이터와 출력 데이터 생성
X = []
y = []
time_steps = 2  # 시계열 길이 설정

for i in range(len(scaled_data) - time_steps):
    X.append(scaled_data[i:i + time_steps])
    y.append(scaled_data[i + time_steps, :])  # 모든 공실률 값 예측

X = np.array(X)
y = np.array(y)

# 모델 구성
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=y.shape[1]))  # 모든 행정동의 공실률을 예측하도록 출력 뉴런 수 조정

model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습과정에서 loss를 저장하기 위해 history 콜백 사용
history = model.fit(X, y, epochs=100, batch_size=32, validation_split=0.2)

# 모델 예측
predicted_vacancy_rates = model.predict(X)

# 예측된 값을 원래 스케일로 되돌리기 위한 준비
# 예측된 값에 빈 컬럼을 추가하여 원래 피처 수와 동일하게 맞춤
predicted_vacancy_rates_full = np.zeros((predicted_vacancy_rates.shape[0], scaled_data.shape[1]))
predicted_vacancy_rates_full[:, :predicted_vacancy_rates.shape[1]] = predicted_vacancy_rates

# 예측된 값을 원래 스케일로 되돌림
predicted_vacancy_rates_inversed = scaler.inverse_transform(predicted_vacancy_rates_full)

# 첫 번째 피처만 가져오기 (공실률 예측 결과)
predicted_vacancy_rates_final = predicted_vacancy_rates_inversed[:, :y.shape[1]]

In [199]:
# 학습 과정 시각화 (loss 및 val_loss)
loss_trace = go.Scatter(
    x=list(range(1, len(history.history['loss']) + 1)),
    y=history.history['loss'],
    mode='lines',
    name='Training Loss'
)

val_loss_trace = go.Scatter(
    x=list(range(1, len(history.history['val_loss']) + 1)),
    y=history.history['val_loss'],
    mode='lines',
    name='Validation Loss'
)

layout = go.Layout(
    title='Model Loss',
    xaxis=dict(title='Epoch'),
    yaxis=dict(title='Loss'),
    legend=dict(x=0, y=1)
)

fig = go.Figure(data=[loss_trace, val_loss_trace], layout=layout)

# Display plot
fig.show()

In [191]:
# Assuming column names correspond to the vacancy rate features
vacancy_columns = ['고운동', '아름동', '종촌동', '도담동', '어진동', '다정동', '새롬동', '한솔동', '보람동', '소담동', '나성동', '반곡동', '해밀동']

# Extracting the last two rows for 2024 and 2025
predicted_2024_2025 = predicted_vacancy_rates_final[-2:, :len(vacancy_columns)]

# Creating a DataFrame with appropriate year labels
predicted_df = pd.DataFrame(predicted_2024_2025, columns=vacancy_columns)
predicted_df['Year'] = [2024, 2025]

# Rearranging columns to have 'Year' first
predicted_df = predicted_df[['Year'] + vacancy_columns]

# Rounding the predictions to two decimal places
predicted_df = predicted_df.round(2)
predicted_df

,Year,고운동,아름동,종촌동,도담동,어진동,다정동,새롬동,한솔동,보람동,소담동,나성동,반곡동,해밀동
0,2024,24.98,37.28,13.70,12.00,24.54,17.82,8.01,55.72,55.53,20.19,44.77,39.09,43.73
1,2025,30.47,23.39,11.85,15.75,23.34,11.91,4.11,30.48,60.07,25.20,35.37,35.91,32.61


In [203]:
predicted_df.to_csv("predicted_vacancy_rates.csv", index=False)